# Script for ```Testing``` ( Most common class for all sub-images of one fish )

In [1]:
import os
import sys
import re
import traceback
from copy import deepcopy
from typing import List, Dict
from datetime import datetime
from glob import glob
import json
import yaml
from collections import Counter

from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import cv2

import torch
from torch import nn, utils
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad


sys.path.append(r"C:\Users\confocal_microscope\Desktop\ZebraFish_AP_POS\modules") # add path to scan customized module
from logger import init_logger
from fileop import create_new_dir
from dl_utils import set_gpu, ImgDataset, caulculate_metrics, save_model, plot_training_trend, \
                     confusion_matrix_with_class, get_sortedClassMapper_from_dir
from cam_utils import reshape_transform, postprocess_cam_image

# print("="*100, "\n")

In [2]:
testingByFish_logger = init_logger(r"Testing by fish")

Constant path

In [3]:
dataset_root = r"C:\Users\confocal_microscope\Desktop\{Test}_DataSet"
load_dir_root = r"C:\Users\confocal_microscope\Desktop\{Test_by_fish}_Model_history"

Args

In [4]:
dataset_name = r"{20230305_NEW_STRUCT}_Academia_Sinica_i409"
dataset_gen_method = "fish_dataset_horiz_cut_1l2_Mix_AP"
dataset_param_name = "DS_SURF3C_CRPS256_SF14_INT20_DRP70_RS2022"
cuda_idx = 1
batch_size = 16
model_name = "vit_b_16"
model_history = r"20230404_11_36_06_{EarlyStop}_{54_epochs_AugOnFly}"
model_desc = "best" # best / final
use_hsv = False # using 'HSV' when getting images from the 'ImgDataset'
do_cam = True

debug_mode = False
rand_seed = 2022 # only for debug_mode


# Create path var
dataset_dir = os.path.join(dataset_root, dataset_name, dataset_gen_method, dataset_param_name)
test_selected_dir = os.path.join(dataset_dir, "test", "selected")
load_dir = os.path.join(load_dir_root, model_name, model_history)
cam_result_root = os.path.join(load_dir, "GradCAM_result")

# Set GPU
device, device_name = set_gpu(cuda_idx)
testingByFish_logger.info(f"Using '{device}', device_name = '{device_name}'")

| 2023-04-07 01:28:50,719 | Testing by fish | INFO | Using 'cuda', device_name = 'NVIDIA GeForce RTX 2080 Ti'


In [5]:
# Get datetime
time_stamp = datetime.now().strftime('%Y%m%d_%H_%M_%S')


# Set 'np.random.seed'
np.random.seed(rand_seed)


# Scan classes to create 'class_mapper'
num2class_list, class2num_dict = get_sortedClassMapper_from_dir(test_selected_dir)
testingByFish_logger.info(f"num2class_list = {num2class_list}, class2num_dict = {class2num_dict}")


# Scan tiff
test_img_list = glob(os.path.normpath(f"{test_selected_dir}/*/*.tiff"))
testingByFish_logger.info(f"total = {len(test_img_list)}")
## debug mode: random select 200 images
if debug_mode:
    test_img_list = np.random.choice(test_img_list, size=200, replace=False)
    testingByFish_logger.info(f"Debug mode, only select first {len(test_img_list)}")


# Save 'testing_amount'
testing_amount = f"{{ datatest_{len(test_img_list)} }}_{{ test_{len(test_img_list)} }}"
with open(os.path.normpath(f"{load_dir}/{testing_amount}"), mode="w") as f_writer: pass


# Create 'test_set', 'test_dataloader'
testingByFish_logger.info(f"test_data ({len(test_img_list)})")
[testingByFish_logger.info(f"{i} : img_path = {test_img_list[i]}") for i in range(5)]
test_set = ImgDataset(test_img_list, class_mapper=class2num_dict, resize=(224, 224), 
                      use_hsv=use_hsv)
test_dataloader = DataLoader(test_set, batch_size=batch_size, shuffle=False, pin_memory=True)
testingByFish_logger.info(f"※ : total test batches: {len(test_dataloader)}")


# Read test ( debug mode only )
if debug_mode:
    read_test = cv2.imread(test_img_list[-1])
    testingByFish_logger.info(f"Read Test: {test_img_list[-1]}")
    cv2.imshow("Read Test", read_test)
    cv2.waitKey(0)


# Create model
testingByFish_logger.info((f"load model using 'torch.hub.load()', "
                     f"model_name: '{model_name}', weights: '{model_name}/{model_history}/{model_desc}_model.pth'"))
model = torch.hub.load('pytorch/vision', model_name, weights=None)
## modify model structure
model.heads.head = nn.Linear(in_features=768, out_features=len(class2num_dict), bias=True)
model.to(device)
# print(model)
## load 'model_state_dict'
model_path = os.path.join(load_dir, f"{model_desc}_model.pth")
pth_file = torch.load(model_path, map_location=device) # unpack to device directly
model.load_state_dict(pth_file["model_state_dict"])


""" Initial 'CAM' generator

Explain: https://github.com/jacobgil/pytorch-grad-cam/blob/master/tutorials/vision_transformers.md

Code-Ref: https://github.com/jacobgil/pytorch-grad-cam/blob/2183a9cbc1bd5fc1d8e134b4f3318c3b6db5671f/usage_examples/vit_example.py#L102
使用 torch.hub.load('facebookresearch/deit:main','deit_tiny_patch16_224', pretrained=True) 時 target_layers = [model.blocks[-1].norm1]

透過 print(model) 比較後 torch.hub.load('pytorch/vision', vit_b_16, weights=None) 應使用 target_layers = [model.encoder.layers.encoder_layer_10.ln_1]

"""
if do_cam:
    target_layers = [model.encoder.layers.encoder_layer_10.ln_1]
    cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True, reshape_transform=reshape_transform)
    cam.batch_size = batch_size


# Testing
## testing variable
test_log = { "Test": time_stamp, "model_desc": f"{model_desc}_model.pth" }
fish_gt_dict = {}
fish_predcnt_dict = {}
## progress bar
pbar_n_test = tqdm(total=len(test_dataloader), desc="Test ")
## start testing
## set to evaluation mode
model.eval()
for batch, data in enumerate(test_dataloader):
    x_test, y_test, fish_name_batch = data
    x_test, y_test = x_test.to(device), y_test.to(device) # move to GPU
    preds = model(x_test)
    _, pred_test = torch.max(preds, 1)
    
    
    ## generate 'CAM' heatmap
    ### 'ImgDataset' converts the 'img_dims' to the format required by the model, so it needs to be converted back.
    ### usage: np.moveaxis([ batch_size, C, H, W ] , 1, -1) --> [ batch_size, H, W, C ]
    if do_cam:
        targets = None # If None, returns the map for the highest scoring category. Otherwise, targets the requested category.
        grayscale_cam_batch = cam(input_tensor=x_test, targets=targets, aug_smooth=True, eigen_smooth=True)
        rgb_img_batch = np.moveaxis(deepcopy(x_test).cpu().numpy(), 1, -1)
    
    
    ## update 'predict_class' according to 'fish_name'
    pred_test_list = pred_test.cpu().numpy().tolist()
    for i, fish_name_tuple in enumerate(zip(*fish_name_batch)):
        
        ### update 'fish_gt_dict'
        fish_name_list = list(fish_name_tuple)
        fish_name = "_".join(fish_name_list[:4]) # example_list : ['L', 'fish', '111', 'A', 'selected', '0']
                                                 # list[:3] = L_fish_111 ; list[:4] = L_fish_111_A
        if fish_name not in fish_gt_dict: fish_gt_dict[fish_name] = fish_name_list[0]
        
        ### update 'fish_predcnt_dict'
        if fish_name not in fish_predcnt_dict: fish_predcnt_dict[fish_name] = Counter()
        fish_predcnt_dict[fish_name].update([num2class_list[pred_test_list[i]]])
        
        
        ### show cam on image
        if do_cam:
            cam_result_dir = os.path.join(cam_result_root, fish_name)
            create_new_dir(cam_result_dir, display_in_CLI=False)
            fish_name_list[4] = "cam"
            cam_save_path = os.path.normpath(f"{cam_result_dir}/{'_'.join(fish_name_list)}.tiff")
            with open(os.path.join(dataset_dir, "dataset_config.yaml"), mode="r") as f_reader: 
                dataset_config = yaml.load(f_reader, Loader=yaml.SafeLoader)
            postprocess_cam_image(rgb_img_batch[i], grayscale_cam_batch[i, :], 
                                  resolution=dataset_config["gen_param"]["crop_size"],
                                  pred_cls=num2class_list[pred_test_list[i]],
                                  gt_cls=fish_name_list[0],
                                  font_size=20,
                                  cam_save_path=cam_save_path)
    
    
    ## show predict_status of current_batch in CLI
    testingByFish_logger.info((f"Batch[ {(batch+1):0{len(str(len(test_dataloader)))}} / {len(test_dataloader)} ], "
                                f"# of (ground truth == prediction) in_this_batch : "
                                f"{(pred_test.cpu() == y_test.cpu()).sum().item():{len(str(len(y_test)))}}/{len(y_test)}"))
    
    ## update 'pbar_n_test'
    pbar_n_test.update(1)
    pbar_n_test.refresh()


# TODO:  add 'pred_log' in for each crop
# TODO:  add 'pred_log' in for each fish

for key, value in fish_predcnt_dict.items(): fish_predcnt_dict[key] = value.most_common(1)[0][0]
pred_list_to_name = [ value for _, value in fish_predcnt_dict.items() ]
gt_list_to_name = [ value for _, value in fish_gt_dict.items() ]

caulculate_metrics(test_log, None,
                   gt_list_to_name, pred_list_to_name, class2num_dict)
# print(json.dumps(test_log, indent=4))
pbar_n_test.close()
## end testing


# Save infomations to a file
with open(os.path.normpath(f"{load_dir}/{{Logs}}_test.log"), mode="w") as f_writer:

    ## change direction of 'sys.stdout'
    orig_stdout = sys.stdout # store original 'sys.stdout'
    sys.stdout = f_writer

    ## write 'test_log'
    print(json.dumps(test_log, indent=4), "\n\n")

    ## write 'classification_report'
    cls_report = classification_report(y_true=gt_list_to_name, y_pred=pred_list_to_name)
    print("Classification Report:\n\n", cls_report, "\n")

    ## write 'confusion_matrix'
    #   row: Ground truth
    #   column: predict
    #  *　0　1　2
    #  0 [] [] []
    #  1 [] [] []
    #  2 [] [] []
    #
    confusion_mat = confusion_matrix_with_class(ground_truth=gt_list_to_name, prediction=pred_list_to_name)

    ## recover direct of 'sys.stdout'
    sys.stdout = orig_stdout


# Rename 'load_dir'
## new_name_format = {time_stamp}_{state}_{target_epochs_with_ImgLoadOptions}_{test_f1}
## state = {EarlyStop, Interrupt, Completed, Tested, etc.}
model_history_list = re.split("{|}", model_history)
new_name = f"{model_history_list[0]}{{Tested}}_{{{model_history_list[3]}}}_{{{model_desc}}}_{{avg_f1_{test_log['average_f1']}}}" 
os.rename(load_dir, os.path.join(load_dir_root, model_name, new_name))

| 2023-04-07 01:28:50,774 | Testing by fish | INFO | num2class_list = ['L', 'M', 'S'], class2num_dict = {'L': 0, 'M': 1, 'S': 2}
| 2023-04-07 01:28:50,896 | Testing by fish | INFO | total = 23126
| 2023-04-07 01:28:50,897 | Testing by fish | INFO | test_data (23126)
| 2023-04-07 01:28:50,898 | Testing by fish | INFO | 0 : img_path = C:\Users\confocal_microscope\Desktop\{Test}_DataSet\{20230305_NEW_STRUCT}_Academia_Sinica_i409\fish_dataset_horiz_cut_1l2_Mix_AP\DS_SURF3C_CRPS256_SF14_INT20_DRP70_RS2022\test\selected\L\L_fish_111_A_selected_0.tiff
| 2023-04-07 01:28:50,898 | Testing by fish | INFO | 1 : img_path = C:\Users\confocal_microscope\Desktop\{Test}_DataSet\{20230305_NEW_STRUCT}_Academia_Sinica_i409\fish_dataset_horiz_cut_1l2_Mix_AP\DS_SURF3C_CRPS256_SF14_INT20_DRP70_RS2022\test\selected\L\L_fish_111_A_selected_1.tiff
| 2023-04-07 01:28:50,899 | Testing by fish | INFO | 2 : img_path = C:\Users\confocal_microscope\Desktop\{Test}_DataSet\{20230305_NEW_STRUCT}_Academia_Sinica_i409\fi

Test :   0%|          | 0/1446 [00:00<?, ?it/s]

| 2023-04-07 01:29:09,746 | Testing by fish | INFO | Batch[ 0001 / 1446 ], # of (ground truth == prediction) in_this_batch : 16/16
| 2023-04-07 01:29:24,089 | Testing by fish | INFO | Batch[ 0002 / 1446 ], # of (ground truth == prediction) in_this_batch : 15/16
| 2023-04-07 01:29:38,393 | Testing by fish | INFO | Batch[ 0003 / 1446 ], # of (ground truth == prediction) in_this_batch : 16/16
| 2023-04-07 01:29:53,826 | Testing by fish | INFO | Batch[ 0004 / 1446 ], # of (ground truth == prediction) in_this_batch : 16/16
| 2023-04-07 01:30:09,586 | Testing by fish | INFO | Batch[ 0005 / 1446 ], # of (ground truth == prediction) in_this_batch : 15/16
| 2023-04-07 01:30:25,147 | Testing by fish | INFO | Batch[ 0006 / 1446 ], # of (ground truth == prediction) in_this_batch : 14/16
| 2023-04-07 01:30:39,725 | Testing by fish | INFO | Batch[ 0007 / 1446 ], # of (ground truth == prediction) in_this_batch : 14/16
| 2023-04-07 01:30:53,877 | Testing by fish | INFO | Batch[ 0008 / 1446 ], # of (gr